In [1]:


import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url2 = 'https://cocl.us/Geospatial_data'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table')

postal_codes = []
Borough = []
Neighborhood = []

for table in tables:
    rows = table.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')

        if len(cells) > 1:
            rank = cells[0]
            rr = rank.text.strip()
            if "M" in rr:
                postal_codes.append(rank.text.strip())
            else:
                continue

            country = cells[1]
            Borough.append(country.text.strip())

            rate = cells[2]
            Neighborhood.append(rate.text.strip())

postal_codes.pop()
Borough.pop()
Neighborhood.pop()

idx = list(range(len(postal_codes)))
df = pd.DataFrame(postal_codes, index=idx, columns = ['PostalCode'])

df['Borough'] = Borough
df['Neighborhood'] = Neighborhood
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df = df.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
mask = df['Neighborhood'] == "Not assigned"
df.loc[mask, 'Neighborhood'] = df.loc[mask, 'Borough']

coords = pd.read_csv(url2)
coords.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
df = pd.merge(df, coords, on='PostalCode', how='inner')
df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
